In [1]:
import numpy as np
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.utils import plot_model
from keras import backend as K

import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


레이블 개수 계산

In [3]:
num_labels = len(np.unique(y_train))
num_labels

10

이미지 차원

In [5]:
image_size = x_train.shape[1]
image_size

28

크기 조정, 정규화

In [6]:
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

네트워크 매개변수

In [7]:
input_shape = (image_size, image_size,1)
batch_size = 32
kernel_size = 3
latent_dim = 16

cnn 계층당 필터의 인코더/디코더 개수

In [8]:
layer_filters = [32, 64]

오토인코더 모델 구성

인코더 모델 구성

In [9]:
inputs = Input(shape = input_shape, name='encoder_input')

In [11]:
x = inputs
for filters in layer_filters:
  x = Conv2D(filters = filters, kernel_size=kernel_size, activation='relu', strides=2, padding='same')(x)

디코더 모델 구성에 필요한 형상 정보


형상은 7,7,64 로 디코더에 의해 처리되면 28,28,1로 복구

In [12]:
shape = K.int_shape(x)

In [13]:
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

In [14]:
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 latent_vector (Dense)       (None, 16)                50192     
                                                                 
Total params: 69,008
Trainable params: 69,008
Non-trainable params: 0
_________________________________________________________________


디코더 모델 구성

In [15]:
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

In [16]:
for filters in layer_filters[::-1]:
  x = Conv2DTranspose(filters=filters, kernel_size=kernel_size, activation='relu', strides=2, padding='same')(x)

In [17]:
outputs = Conv2DTranspose(filters=1, kernel_size=kernel_size, activation='sigmoid', padding='same', name='decoder_output')(x)

In [18]:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 3136)              53312     
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 decoder_output (Conv2DTrans  (None, 28, 28, 1)        289 

In [19]:
autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                69008     
                                                                 
 decoder (Functional)        (None, 28, 28, 1)         108993    
                                                                 
Total params: 178,001
Trainable params: 178,001
Non-trainable params: 0
_________________________________________________________________


In [20]:
autoencoder.compile(loss='mse', optimizer='adam')

In [21]:
autoencoder.fit(x_train, x_train, validation_data=(x_test, x_test), epochs=1, batch_size=batch_size)

1875/1875 [==============================] - 152s 81ms/step - loss: 0.0211 - val_loss: 0.0107


In [22]:
x_decoded = autoencoder.predict(x_test)

313/313 [==============================] - 6s 19ms/step


In [22]:
imgs = np.concatenate([x_test[:8], x_decoded[:8]])
imgs = imgs.reshape((4,4,image_size, image_size))
imgs = np.vstack([np.,hstack(i) for i in imgs])
plt.figure()
plt.axis('off')
plt.title('Inpu: 1st 2rows, Decoded: last 2 rows')
plt.imshow(imgs, interpolation='none', cmap='gray')
plt.show()